# Data Analysis for AER I in cycle 501
***

## Constants and functions

In [1]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import scipy.constants as sc
import struct
import scipy.optimize as opt
import glob
from mpl_toolkits.mplot3d import Axes3D
import os
import csv
import math

## SPICE analysis
***

In [ ]:
def get_pol(fr):
    """Converts fliping ratio to polarization."""
    return (fr - 1)/(1 + fr)

def get_scan(number):
    """Returns the data file scan number in Spice format."""
    const = 3
    if (number > 999):  #no leading zeros
        zeros = 0
    else:
        if (number == 0):
            zeros = const
        else:
            zeros = const - int(math.floor(math.log10(number)))
    return '0'*zeros + str(number)

def load_data(path, names, dest, write_data=False, overwrite=False, reorder=False):
    """Returns the list [scan number, scan date, scan time start, user input scan titles, matrix of data from scan]."""
    scan_nums,dates,times,scan_titles,parms,data =[[],[],[],[],[],[]]
    for i in range(len(names)):
        file = open(path + names[i])
        lines = file.readlines()
        lines = [i.replace('\n','') for i in lines]  #removes new line characters
        file.close()
        
        scan_nums.append(lines[0][2::])  #line number determined by Spice format
        scan_nums[i] = scan_nums[i].split(' = ',)[1]
        dates.append(lines[1][2::])
        dates[i] = dates[i].split(' = ',)[1]
        times.append(lines[2][2::])
        times[i] = times[i].split(' = ',)[1]
        scan_titles.append(lines[10][2::])
        scan_titles[i] = scan_titles[i].split(' = ',)[1]
        parms.append(lines[29][7::].split())
        parms[i].append('count rate')
        data.append([j.split()[1::] for j in lines[30::] if (j[0] != '#')])    #assigns the data
        data[i] = [[float(data[i][j][k]) for j in range(len(data[i]))] for k in range(len(data[i][0]))]#HERE'S THE BUG    #converts to float
        data[i] = [list(j) for j in zip(*data[i])]    #transposes table
        
        time_normed_counts = [data[i][j][2]/data[i][j][1] if (data[i][j][1] != 0) else 0 for j in range(len((data[i])))]
        for j in range(len(data[i])):
            data[i][j].append(time_normed_counts[j])
        
        if ((reorder == True) and (parms[i] != default_order)):    #rearranges the columns to default_order
            swap_i = []    #columns to swap
            for j in range(len(parms[i])):
                swap_i.append([j, default_dict[parms[i][j]]])

            swaped_data = [ [0]*len(data[i][0]) for _ in range(len(data[i]))]    #matrix to hold swaped values
            for j in range(len(data[i])):    #slow swapping method, may need some work...
                for k in range(len(data[i][j])):
                    for m in range(len(swap_i)):
                        if (swap_i[m][1] == k):
                            swaped_data[j][k] = data[i][j][m]
                            parms[i][m] = default_order[swap_i[m][0]]    
            data[i] = swaped_data
            
        if (write_data == True):    #writes data to csv files
            if (overwrite == True):
                c_str = 'w'
            else:
                c_str = 'x'    #Fails if the file already exists
            windows_fname = scan_titles[i]    #Cleans the Spice file names for Windows
            invalid = '<>:"/\|?*'
            for char in invalid:
                windows_fname = windows_fname.replace(char,'')
            with open(dest_path + "scan" + str(scan_nums[i]) + "_" + windows_fname + ".csv", c_str, newline='') as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow([scan_titles[i]])
                writer.writerow([dates[i]])
                writer.writerow([times[i]])
                writer.writerow(parms[i])
                for j in range(len(data[i])):
                    writer.writerow(data[i][j])
    
    return scan_nums,dates,times,scan_titles,parms,data

In [ ]:
data_path = "C:/SpICE/User/exp9/Datafiles/"
dest_path = "C:/Users/User/Desktop/AER_501/"  #WARNING: may overwrite files in destination folder!

default_order = ['detector','time','rate','nu_pre','nu_post','phase','prism','s_tran','s_rot', 'a_tran','a_rot']
default_dict = {i[1]:i[0] for i in enumerate(default_order)}

In [ ]:
file_range = range(59,61) #doesn't include endpoint!
file_list = ["CG4B_exp0009_scan" + get_scan(i) + ".dat" for i in file_range]
Num,Date,Time,Title,Parms,Data = load_data(data_path,file_list,dest_path,write_data=False,overwrite=False,reorder=True)

## Anger reduction
***

In [ ]:
#limits for filtering data outside of the image
imgpar = [[0,1500],[0,1500]]
#imgpar = [[200,350],[150,300]]
spatialscale = 512./116.  #what is this?

def loadData(openfile, imgdim):
    """Returns raw data and BG data, filtered by the imgdim and BG dim."""
    listdata = []
    BGdata = []
    maxt = 0
    dummy = np.zeros(shape=(3))
    with open(openfile, "rb") as f:
        bytesin = f.read(24)
        while bytesin:
            temp = struct.unpack('3d', bytesin)
            dummy[0] = temp[0]
            dummy[1] = temp[1] * spatialscale
            dummy[2] = temp[2] * spatialscale
            listdata.append([dummy[0], int(dummy[1]-imgpar[0][0]), int(dummy[2]-imgpar[1][0])])
            
            bytesin = f.read(24)
    return listdata, BGdata

#This framestack is full image (spatially integrated)
#Think the raw data format is time in ticks (100ns/tick), position scaled to some length
def framestack(inarray, period, Nperiods, nbins):
    t_hist = np.zeros(shape=(nbins))
    binsize = period / nbins
    dlen = len(inarray)
    curindex = 0
    for i in range(0, dlen):
        curindex = int(np.floor( (inarray[i][0] % period) / binsize ))
        t_hist[curindex] += 1
    plt.plot(t_hist)
    return

def plotTotal(inarray):
    t_hist = np.zeros(shape=(int(max(map(lambda x: x[0], inarray)))+1))
    dlen = len(inarray)
    curindex = 0
    for i in range(0, dlen):
        curindex = int(np.floor( (inarray[i][0])))
        t_hist[curindex] += 1
    plt.plot(t_hist)
    return
    
def BGsubtractIMG(inImg, BG):
    ydim = int(max(map(lambda x: x[1], BG)) +1)
    xdim = int(max(map(lambda x: x[2], BG)) + 1)
    BGave = len(BG)/ (ydim * xdim)
    result = np.array(inImg)
    xlen = len(inImg)
    ylen = len(inImg[0])
    for i in range(0, xlen):
        for j in range(0,ylen):
            result[i][j] -= BGave
    return result

def ShowImage(inImage):
    fig, ax = plt.subplots(1,1, figsize=(8,6))
    im = ax.imshow(inImage, origin="lower", aspect="auto")
    fig.colorbar(im)
    fig.tight_layout()
    plt.show()
    
def MakeImage(inarray):
    ydim = int(max(map(lambda x: x[1], inarray)) + 1)
    xdim = int(max(map(lambda x: x[2], inarray)) + 1)
    img = np.zeros(shape=(xdim, ydim))
    dlen = len(inarray)
    for i in range(0, dlen):
        xind = int(inarray[i][1])
        yind = int(inarray[i][2])
        img[yind][xind] += 1
    
    return img

#This function combines bins in time, need to take care to change scale when using
def BinDataT(inData, n):
    bSize = n
    result = np.array(inData)
    nData = len(inData)
    for i in range(0, nData):
        result[i][0] = np.floor(result[i][0] / bSize)
    return result
    
#This function bins spatial dimensions, need to take care to change scale when using
def BinImg(inImg, xbinsize, ybinsize):
    xdim = len(inImg[0])
    ydim = len(inImg)
    ybins = int(np.floor(ydim/ybinsize))
    xbins = int(np.floor(xdim/xbinsize))
    result = np.zeros(shape=(ybins, xbins))
        
    for xb in range(0, xbins):
        curx = xb * xbinsize
        for yb in range(0, ybins):
            cury = yb * ybinsize
            for x in range(curx, curx + xbinsize):
                for y in range(cury, cury+ybinsize):
                    result[yb][xb] += int(inImg[y][x])
    return result
    
def BinRaw(inData, xbinsize, ybinsize):
    result = []
    #lastXbin = np.floor(max(map(lambda x: x[1], inData)) / xbinsize)
    #lastYbin = np.floor(max(map(lambda x: x[2], inData)) / ybinsize)
    for datum in inData:
        newXbin = datum[1]/xbinsize
        newYbin = datum[2]/ybinsize
        #if (newXbin > lastXbin):
        #    continue
        #if (newYbin > lastYbin):
        #    continue
        result.append([datum[0], int(np.floor(newXbin)), int(np.floor(newYbin))])
    return np.array(result)

def fitfun(x, a, f, phase, c):
    return c + a * np.sin(2*sc.pi*f *x + phase)

def tfit(tData, par0):
    t = np.linspace(0, len(tData), len(tData))
    #frequency estimate is in kHz, converted to pixels
    par0[0] = np.std(tData)
    par0[3] = np.mean(tData)
    popt, pcov = opt.curve_fit(fitfun, t, tData, p0 = par0, maxfev=1000000,ftol=1e-8)
    tt = np.linspace(0, len(tData), 1000)
    plt.plot(tt, fitfun(tt, *popt), linestyle="-", color='b')
    plt.plot(tData, linestyle="None", marker="o", color='r')
    plt.show()
    return popt

def T_analysis_byPixel(inData, par0):
    maxt = int(max(map(lambda x: x[0], inData)))
    #using hash table to uniquely identify each pixel 
    maxx = int(max(map(lambda x: x[1], inData)))
    maxy = int(max(map(lambda x: x[2], inData)))
    hashmax = maxx*(maxy + 1) + maxy
    #hash: i_x*(n_y - 1) + i_y (or vice versa)
    #iterate through all data once, separating into different hash groups.
    #then analyze the hash lists separately
    hashlist = np.zeros(shape=(hashmax+1, maxt+1), dtype=np.int32)
    for datum in inData:
        curhash = int(datum[1]*(maxy + 1) + datum[2])
        hashlist[curhash][int(datum[0])] += 1
    imgOut = np.zeros(shape=(maxy+1, maxx+1))
    for i in range(0, hashmax):
        tlist = hashlist[i]
        pixeldata = tfit(tlist, par0)
        xpix = int(np.floor((i / (maxy + 1)) ))
        ypix = int(i % (maxy + 1))
        #currently outputs in degrees
        imgOut[ypix][xpix] = 360*pixeldata[2]/(2.*sc.pi) #select which item from the fit you want to view
    return imgOut

def totalCounts(inData):
    return len(inData)

def scanCounts(f_i, f_f, imgdim, BGdim, directory):
    result = []
    filelist = []
    filenum = ""
    filenumlist = []
    configlist = []
    for filename in glob.glob(directory+"MURR*.dat"):
        filenum = filename[filename.find("MURR")+4:filename.find(".dat")]
        if (int(filenum) >= f_i and int(filenum) <= f_f):
            filelist.append(filename)
            filenumlist.append(filenum)
            configlist.append(getConfig(filenum, directory))
    for filename in filelist:
        data, BG = loadData(filename, imgdim, BGdim)
        BGperPix = totalCounts(BG) / ((BGdim[0][1] - BGdim[0][0]) * (BGdim[1][1] - BGdim[1][0]))
        rawcount = totalCounts(data)
        finalcount = rawcount - BGperPix * ((imgdim[0][1] - imgdim[0][0]) * (imgdim[1][1] - imgdim[1][0]))
        result.append(finalcount)
    return filenumlist, np.array(configlist), np.array(result)
 
#easier if runID is input as a string
#Big run did 2D scan of B2 and G2, try 3D plot or heatmap type plot
def getConfig(runID, direct):
    result = []
    for line in open(direct + "logfile.txt", "r"):
        element = line.find("MURR" + runID)
        if element > 0:
            result.append(round(float(line[line.find("__B1")+5:line.find("_B2")]) + 1e-4, 2)) #append B1 value
            result.append(round(float(line[line.find("_B2")+4:line.find("_G1")])+ 1e-4, 2)) #append B2 value
            result.append(round(float(line[line.find("_G1")+4:line.find("_G2")])+ 1e-4, 2)) #append G1 value
            result.append(round(float(line[line.find("_G2")+4:line.find("_N1")])+ 1e-4, 2)) #append G2 value
            result.append(round(float(line[line.find("_N1")+4:line.find("_N2")])+ 1e-4, 2)) #append N1 value
            result.append(round(float(line[line.find("_N2")+4:line.find("_RF1Fre")])+ 1e-4, 2)) #append N2 value
            result.append(round(float(line[line.find("_RF1Fre")+8:line.find("_RF1Amp")])+ 1e-4, 2)) #append RF1Freq value
            result.append(round(float(line[line.find("_RF1Amp")+8:line.find("_RF1phase")])+ 1e-4, 2)) #append RF1Amp value
            result.append(round(float(line[line.find("_RF1phase")+10:line.find("_RF2Fre")])+ 1e-4, 2)) #append RF1phase value
            result.append(round(float(line[line.find("_RF2Fre")+8:line.find("_RF2Amp")])+ 1e-4, 2)) #append RF2Freq value
            result.append(round(float(line[line.find("_RF2Amp")+8:line.find("_RF2phase")])+ 1e-4, 2)) #append RF2Amp value
            result.append(round(float(line[line.find("_RF2phase")+10])+ 1e-4, 2)) #append RF2phase value
            return result
    return result
            
def plot2dparam(configlist, countlist, n1, n2):
    histx, histy = convertarraystohist(configlist[:,n1], configlist[:,n2], countlist)
    h, xe, ye, im = plt.hist2d(histx, histy, bins=[50,27])
    plt.clf()
    plt.close()
    ext = [xe[0], xe[-1], ye[0], ye[-1]]
    fig, ax = plt.subplots(1,1, figsize=(8,6))
    plt.imshow(h.T, extent=ext, origin="lower", interpolation="None", aspect='auto')
    cb = fig.colorbar(im)
    cb.ax.tick_params(labelsize=16)
    plt.tick_params(which='both', labelsize=16)
    ax.set_ylabel("$B_{rf}$ [Vpp]", fontsize=18)
    #plt.yticks(np.linspace(0,len(ylab),4), np.round(np.array([ylab[0],ylab[int(len(ylab)/3)], ylab[int(len(ylab)*2/3)], ylab[len(ylab)-1]]),2), fontsize=16)
    ax.set_xlabel("Gradient field [A]", fontsize=18)
    fig.tight_layout()
    plt.show()
    
def convertarraystohist(x, y, z):
    outx = []
    outy = []
    ndata = len(z)
    for i in range(0, ndata):
        for j in range(0, int(z[i])):
            outx.append(x[i])
            outy.append(y[i])
    return outx, outy

def convertarraystoimg(x, y, z):
    sort1 = sorted(set(x))
    sort2 = sorted(set(y))
    stepx = round(abs(sort1[1] - sort1[0]),2)
    stepy = round(abs(sort2[1] - sort2[0]),2)
    minx = int(min(x)*100)
    miny = int(min(y)*100)
    result = np.zeros(shape=(len(sort2), len(sort1)))
    #flipping x and y d/t image being row/col opposite of standard x/y notation
    xlabel = np.round(np.copy(sort1),2)
    ylabel = np.round(np.copy(sort2),2)
    ndata = len(z)
    for i in range(0, ndata):
        result[int(np.floor((y[i]*100-miny)/(stepy*100))), int(np.floor((x[i]*100-minx)/(stepx*100)))] = z[i]
    return result, xlabel, ylabel
 
def show2DdataImage(img, xlab, ylab):
    fig, ax = plt.subplots(1,1, figsize=(8,6))
    ext = [xlab[0], xlab[-1], ylab[0], ylab[-1]]
    im = ax.imshow(img, origin="lower", extent=ext, aspect="auto", interpolation="None")
    cb = fig.colorbar(im)
    cb.ax.tick_params(labelsize=16)
    plt.tick_params(which='both', labelsize=16)
    ax.set_ylabel("$B_{rf}$ [Vpp]", fontsize=18)
    #plt.yticks(np.linspace(0,len(ylab),4), np.round(np.array([ylab[0],ylab[int(len(ylab)/3)], ylab[int(len(ylab)*2/3)], ylab[len(ylab)-1]]),2), fontsize=16)
    ax.set_xlabel("Gradient field [A]", fontsize=18)
    fig.tight_layout()
    plt.show()    

def reduce_data(first,last,print_fnames=False, print_raw=False, print_BGsub=False,print_ROI=False,print_summed=False):
    """Returns the summed, BG corrected, ROI 2d array of counts. Includes both starting and ending indicies."""
    
    direct = "C:/Users/Larmor/Dropbox/AngerCamera/Wide_Angle_NSE_2022/"

    binsize = 1  #each pixel is 0.9 mm x 0.9 mm
    
    names = np.arange(first, last + 1, 1, dtype=int)
    filenames = ["CG4B" + str(i) + ".dat" for i in names]
    
    if (print_fnames == True):
        print("Files used:", filenames)
        print("Bin size = ",binsize)
        print("Binned pixel size (mm) = ",  round(binsize*.2,3))
        
    ROIsumlist = np.zeros((len(names)))
    BGsumlist = np.zeros((len(names)))
    BGperpixellist = np.zeros((len(names)))

    ROIImages = []    #stores the background corrected region of interest 2d arrays

    MissingFiles = []
    for i in np.arange(len(names)):
        if os.path.exists(direct+filenames[i]) == True:
            data, BG = loadData(direct+filenames[i], imgpar)
            RawImg = MakeImage(data) 
            BinnedImg = BinImg(RawImg, binsize, binsize)    #returns 2d np array
            
            if print_raw:
                print('Raw Image:')
                ShowImage(BinnedImg)    #NOT BG subtracted!

            zstart = 1 #read from vertical axis of images
            zend = int(480/binsize) #MAGIC, DO NOT CHANGE
            xstart = 1 #read from horizontal axis image 
            xend = int(480/binsize)

            rowrange = np.arange(xstart, xend + 1, 1)
            columnrange = np.arange(zstart, zend + 1, 1)

            BGzstart = 0 #read from vertical axis of images
            BGzend = 40
            BGxstart = 0 #read from horizontal axis image 
            BGxend = 40

            BGrowrange = np.arange(BGxstart, BGxend + 1, 1)
            BGcolumnrange = np.arange(BGzstart, BGzend + 1, 1)

            for c in BGcolumnrange:
                for r in BGrowrange:
                    BGsumlist[i] += BinnedImg[c,r]     #total number of background counts
            numBGpixels = len(BGrowrange)*len(BGcolumnrange)

            BGperpixellist[i] =  BGsumlist[i]/numBGpixels
            
            BGcorrectionxrange = rowrange
            BGcorrectionzrange = columnrange
            for c in BGcorrectionzrange:
                for r in BGcorrectionxrange:
                    BinnedImg[c,r] -= BGperpixellist[i]

            if (print_BGsub == True):
                print("Subscan name: ",filenames[i])
                print("BG per pixel:", BGperpixellist[i])
                ShowImage(BinnedImg)    #BG subtracted

            #Defines the region of interest; MAGIC NUMBERS, DO NOT CHANGE
            ROIzstart = int(80/binsize) #read from vertical axis of images
            ROIzend = int(330/binsize)
            ROIxstart = int(175/binsize) #read from horizontal axis image 
            ROIxend = int(425/binsize)
            
            ROIrowrange = np.arange(xstart, xend + 1, 1)
            ROIcolumnrange = np.arange(zstart, zend + 1, 1)

            for c in ROIcolumnrange:
                for r in ROIrowrange:
                    ROIsumlist[i] += BinnedImg[c,r]
                    
            if print_ROI:
                print("Total counts in ROI: ", ROIsumlist[i], "\n")
                ShowImage(BinnedImg[ROIzstart:ROIzend,ROIxstart:ROIxend])
            
            ROIImages.append(BinnedImg[ROIzstart:ROIzend,ROIxstart:ROIxend])

        else:
            MissingFiles.append(filenames[i])

    summedROI = np.zeros((np.shape(ROIImages[0])))
    ROI_z = np.shape(ROIImages[0])[0]
    ROI_x = np.shape(ROIImages[0])[1]

    for i in range(len(ROIImages)):
        for j in range(ROI_z):
            for k in range(ROI_x):
                summedROI[j][k] += ROIImages[i][j][k]

    Total = sum(sum(summedROI))
    if (print_summed == True):
        print("Total counts in summed ROI: ", Total)
        ShowImage(summedROI)

    if (len(MissingFiles) != 0):
        print("Warning! Missing files: ", MissingFiles)
    
    return summedROI,Total

In [ ]:
scan_num_start = 0
scan_num_end = 1
temp, tot = reduce_data(scan_num_start,scan_num_end,print_fnames=True,print_raw=True,print_BGsub=False,print_ROI=False print_summed=True)

## Anger analysis
***